# **Lab 5: Creación de una VPC e Implementación de una NAT Gateway (arreglar)**

Este laboratorio te guiará paso a paso para crear una **VPC** personalizada con subredes públicas y privadas, configurar las tablas de enrutamiento y desplegar una **NAT Gateway**. Este escenario es común al diseñar arquitecturas seguras en AWS, donde las instancias en subredes privadas pueden acceder a Internet sin exponer su tráfico entrante.

---

**Objetivos del Laboratorio**

1. **Crear una VPC personalizada:** Definir un rango CIDR y establecer el entorno base de red.
2. **Configurar subredes públicas y privadas:** Separar recursos que necesitan acceso directo a Internet (subred pública) de aquellos que deben permanecer aislados (subred privada).
3. **Implementar una NAT Gateway:** Permitir que las instancias en la subred privada accedan a Internet para descargar actualizaciones o dependencias, sin exponerlas directamente.
4. **Verificar la conectividad:** Asegurar que la instancia privada pueda salir a Internet a través de la NAT Gateway, mientras permanece inaccesible desde el exterior.

---

**Instrucciones Detalladas**

**1. Crear una VPC**

1. Inicia sesión en la **Consola de AWS**.
2. Ve a **VPC**.
3. Haz clic en **Your VPCs** y luego en **Create VPC**.
4. Configura la VPC:
    - **Name**: `Lab4-VPC`
    - **IPv4 CIDR block**: `10.0.0.0/16`
    - Deja el resto por defecto.
5. Haz clic en **Create VPC**.

---

**2. Crear Subredes**

1. Ve a **Subnets** y haz clic en **Create subnet**.
2. Crea dos subredes:
    - **Subred pública**:
        - **Name**: `Public-Subnet`
        - **VPC**: `Lab4-VPC`
        - **CIDR block**: `10.0.1.0/24`
        - **AZ**: `us-east-1a` (ejemplo)
    - **Subred privada**:
        - **Name**: `Private-Subnet`
        - **VPC**: `Lab4-VPC`
        - **CIDR block**: `10.0.2.0/24`
        - **AZ**: `us-east-1a` (ejemplo)
3. Haz clic en **Create subnet**.

---

**3. Configurar una Internet Gateway**

1. Ve a **Internet Gateways** y haz clic en **Create internet gateway**.
    - **Name**: `Lab4-IGW`
2. Haz clic en **Create internet gateway**.
3. Adjunta el IGW a la VPC:
    - Selecciona `Lab4-IGW`.
    - **Actions > Attach to VPC**.
    - Selecciona `Lab4-VPC` y confirma.

---

**4. Configurar Tablas de Rutas**

- **Tabla de Rutas Pública**:
    1. Ve a **Route Tables**.
    2. Selecciona la tabla asociada con `Lab4-VPC`.
    3. Renómbrala a `Public-Route-Table`.
    4. Edita las rutas:
        - **Destination**: `0.0.0.0/0`
        - **Target**: Selecciona `Internet Gateway (Lab4-IGW)`
    5. En **Subnet Associations**, asocia `Public-Subnet`.
- **Tabla de Rutas Privada**:
    1. Crea una nueva tabla de rutas (`Create route table`).
        - **Name**: `Private-Route-Table`
        - **VPC**: `Lab4-VPC`
    2. En **Subnet Associations**, asocia `Private-Subnet`.

---

**5. Crear una NAT Gateway**

1. Ve a **NAT Gateways**, haz clic en **Create NAT Gateway**.
2. Configura:
    - **Name**: `Lab4-NAT-Gateway`
    - **Subnet**: `Public-Subnet`
    - **Elastic IP**: Allocate new EIP
3. Haz clic en **Create NAT Gateway`.

---

**6. Actualizar la Tabla de Rutas Privada**

1. Selecciona `Private-Route-Table`.
2. Edita las rutas:
    - **Destination**: `0.0.0.0/0`
    - **Target**: `NAT Gateway (Lab4-NAT-Gateway)`

---

**7. Verificar la Configuración**

1. Lanza dos instancias EC2:
    - **Instancia pública**:
        - **Subred**: `Public-Subnet`.
        - **Security Group**: Permitir SSH desde tu IP.
        - Nombre: `Public-Instance`
    - **Instancia privada**:
        - **Subred**: `Private-Subnet`.
        - **Security Group**: Permitir SSH solo desde `Public-Instance`.
        - Nombre: `Private-Instance`
2. Conéctate a la `Public-Instance` vía SSH.
3. Desde la `Public-Instance`, conéctate a la `Private-Instance` mediante su IP privada.
4. Desde la `Private-Instance`, haz `ping 8.8.8.8` o `curl http://aws.amazon.com`. Deberías tener acceso a Internet a través de la NAT Gateway.

---

**Conclusiones**

- Has creado una VPC con una arquitectura básica: una subred pública para recursos que necesitan acceso directo a Internet, y una subred privada para recursos internos.
- La Internet Gateway permite que la subred pública salga a Internet sin restricciones, mientras que la subred privada depende de la NAT Gateway para salida, manteniendo un entorno más seguro al no exponer instancias privadas directamente.
- La separación de rutas y el uso de NAT Gateway es una práctica común y recomendada para arquitecturas seguras y escalables en AWS.

Con esta experiencia, entiendes mejor cómo funciona el enrutamiento dentro de una VPC y cómo aislar y proteger tus recursos al mismo tiempo que les das acceso controlado a Internet.